In [2]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

In [23]:
files = os.listdir('src/csv')
for i in range(len(files)):
  files[i] = 'src/csv/' + files[i]

frames = dict()
for file in files:
  frames[file] = []
  with open(file, "r") as csvfile:
    reader = csv.reader(csvfile, delimiter=";")
    for row in reader:
      frames[file].append(row)
    
for key in frames.keys():
  frames[key] = np.array(elem).transpose()

frames

{'src/csv/Co.csv': array([['1', '0'],
        ['2', '0'],
        ['3', '1'],
        ...,
        ['2046', '0'],
        ['2047', '0'],
        ['2048', '0']], dtype='<U4'),
 'src/csv/Background.csv': array([['1', '0'],
        ['2', '0'],
        ['3', '1'],
        ...,
        ['2046', '0'],
        ['2047', '0'],
        ['2048', '0']], dtype='<U4'),
 'src/csv/Cs.csv': array([['1', '0'],
        ['2', '0'],
        ['3', '1'],
        ...,
        ['2046', '0'],
        ['2047', '0'],
        ['2048', '0']], dtype='<U4'),
 'src/csv/Na.csv': array([['1', '0'],
        ['2', '0'],
        ['3', '1'],
        ...,
        ['2046', '0'],
        ['2047', '0'],
        ['2048', '0']], dtype='<U4'),
 'src/csv/Eu.csv': array([['1', '0'],
        ['2', '0'],
        ['3', '1'],
        ...,
        ['2046', '0'],
        ['2047', '0'],
        ['2048', '0']], dtype='<U4'),
 'src/csv/Am.csv': array([['1', '0'],
        ['2', '0'],
        ['3', '1'],
        ...,
        ['2046', '0'],
   